## Load the necessary Libraries

In [1]:
import os
import pandas as pd


In [2]:
import os

# Path to the desired directory
desired_directory = r'c:\Users\BasRuhl\OneDrive - Emixa\Documents\XRAYxEmixaHackathon'

# Set the current working directory to the desired directory
os.chdir(desired_directory)

# Print the current working directory to confirm
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\BasRuhl\OneDrive - Emixa\Documents\XRAYxEmixaHackathon


## Load all the files and put them in dataframes and look for mixed data types


In [3]:
import os
import pandas as pd

# Define the directory containing the files
directory = 'data'

# Loop over all files in the directory
for filename in os.listdir(directory):
    # Construct the full file path
    file_path = os.path.join(directory, filename)

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path, sep = ",", low_memory=False)
    # Create a valid Python variable name by removing the file extension and replacing spaces with underscores
    variable_name = os.path.splitext(filename)[0].replace(' ', '_')

    # Dynamically create a variable with the DataFrame
    globals()[variable_name] = df

    # Check for mixed data types in the DataFrame
    mixed_type_columns = []
    for col in df.columns:
        if df[col].apply(type).nunique() > 1:
            mixed_type_columns.append(col)

    # Output the columns with mixed data types
    if mixed_type_columns:
        print(f"DataFrame '{variable_name}' has mixed data types in the following columns:")
        for col in mixed_type_columns:
            print(f"  - {col}")
    else:
        print(f"DataFrame '{variable_name}' has no columns with mixed data types.")


DataFrame 'account' has mixed data types in the following columns:
  - email
  - displayname
  - displayrole
  - displaycompany
DataFrame 'auctionwindow' has mixed data types in the following columns:
  - changedate
DataFrame 'bid' has mixed data types in the following columns:
  - bidlog
  - bidstatus
  - createddate
  - bidremarktrader
  - forcustomeracceptance
DataFrame 'bridgeAuctionwindowStatistics' has no columns with mixed data types.
DataFrame 'bridgeBidContact' has no columns with mixed data types.
DataFrame 'bridgeBidOrder' has no columns with mixed data types.
DataFrame 'bridgeCarOrder' has no columns with mixed data types.
DataFrame 'bridgeContactAccount' has no columns with mixed data types.
DataFrame 'bridgeContactCompany' has no columns with mixed data types.
DataFrame 'bridgeContactTraderstatistic' has no columns with mixed data types.
DataFrame 'bridgeOrderAuctionwindow' has no columns with mixed data types.
DataFrame 'bridgeStatisticsTraderstatistics' has no columns w

## Drop redundant columns


In [6]:
# Select columns in auctionwindow
auctionwindow = auctionwindow[['id', 'startdate', 'enddate', 'changedate', 'count', 'maxorderamount']]

# Select columns in Bid and perform one hot encoding 
bid = bid[['bidvalue', 'bidstatus', 'companyid', 'auctionwindowid', 'basedonvalue', 'orderid']]
bid.loc[:, 'bidstatus'] = bid['bidstatus'].fillna('Unknown')
bid_encoded = pd.get_dummies(bid, columns=['bidstatus'], prefix='bidstatus')

# Select columns in car
car  = car.drop(columns=['id'])

## Dataset for recommender system

In [6]:
# Start with a base DataFrame
dataRecommenderSystem = order.drop(columns=['id'])

# Merge order with car 
dataRecommenderSystem = pd.merge(dataRecommenderSystem, on='orderid', how='inner')

# Merge the resulting DataFrame with bid
dataRecommenderSystem = pd.merge(dataRecommenderSystem, bid, on='orderid', how='inner')

# Merge the resulting DataFrame with bid
dataRecommenderSystem = pd.merge(dataRecommenderSystem, company, on='companyid', how='inner', suffixes=('_BidCarOrder', '_Company'))

#Write the final DataFrame to a CSV file
dataRecommenderSystem.to_csv('datasets/RecommenderSystem.csv', index=False)

#Delete the dataRecommenderSystem and mergedBridge DataFrame to free up memory
del dataRecommenderSystem

## Dataset voor populariteit systeem

In [5]:
# Start with a base DataFrame
dataPopularitySystem = order.rename(columns={'id': 'idOrder'})

# Merge order with car 
dataPopularitySystem = pd.merge(dataPopularitySystem,  on='orderid', how='inner')

# Merge the resulting DataFrame with bid
dataPopularitySystem = pd.merge(dataPopularitySystem, bid.drop(columns=['id']), on='orderid', how='inner')

# Merge the resulting DataFrame with auctionwindow
mergedBridge = pd.merge(bridgeOrderAuctionwindow, auctionwindow.rename(columns={'id': 'idAuctionwindow'}), left_on='xbs$auctionwindowid', right_on='idAuctionwindow', how='inner')
dataPopularitySystem = pd.merge(dataPopularitySystem, mergedBridge, left_on='idOrder', right_on='xbs$orderid', how='inner') #Maybe join via bid, but not sure if all data is kept 

# Merge the resulting DataFrame with statistics
mergedBridge = pd.merge(bridgeAuctionwindowStatistics, statistics.rename(columns={'id': 'idStatistics'}), left_on='xbs$statisticsid', right_on='idStatistics', how='inner')
dataPopularitySystem = pd.merge(dataPopularitySystem, mergedBridge, left_on='idAuctionwindow', right_on='xbs$auctionwindowid', how='inner') #maybe join via?

#Write the final DataFrame to a CSV file
dataPopularitySystem.to_csv('datasets/PopularitySystem.csv', index=False)

#Delete the dataPopularitySystem and mergedBridge DataFrame to free up memory
del dataPopularitySystem
del mergedBridge


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373032 entries, 0 to 373031
Columns: 148 entries, idOrder to isxbsdirectcar
dtypes: float64(31), int64(37), object(80)
memory usage: 421.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1646103 entries, 0 to 1646102
Columns: 164 entries, idOrder to forcustomeracceptance
dtypes: float64(36), int64(41), object(87)
memory usage: 2.0+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588474 entries, 0 to 1588473
Columns: 180 entries, idOrder to maxorderamount
dtypes: float64(37), int64(49), object(94)
memory usage: 2.1+ GB
None
